## Introduction

Transformers are a type of deep learning architecture introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. They have since become a dominant architecture in natural language processing (NLP) tasks due to their effectiveness and efficiency.

![TransformerArchitecture](images//The-Transformer-model-architecture.png)

# Encoder
The encoder processes the input sequence and compresses the information into a 'context' or 'memory' that the decoder can then use. You can think about the encoder as a packing process. When you go on vacation you pack your suitcase neatly so all your things fit. And when you open it up again you have a sense of where everything is. The encoder in the Transformer model is like this packing process. It takes a large amount of information (like a sentence in a certain language) and "packs" it into a more compact representation, capturing the essence of the information.

## 1. Input
The input of a transformer is typically sequence data but the architecture has been adapted for other data types such as images or time series data (e.g. EEG data).

## 2. Embeddings

Transformers rely heavily on the concept of attention. Before we can apply attention or any other operation, we need to represent our input data, typically words or subwords, in a format that a model can understand. This representation is done using embedding vectors. 

***Concept of Word Embeddings:***
Word embeddings are dense vector representations of words in a continuous vector space. The idea is to represent words in such a way that semantically similar words are closer in the vector space.

In [5]:
import torch.nn as nn

vocab_size = 10000  # Example vocabulary size
embedding_dim = 512  # Size of embedding vectors

embedding = nn.Embedding(vocab_size, embedding_dim)

In this code snippet the Embedding layer is created. This layer is a learnable parameter with size 10000 x 512. vocab_size is the amount of words that the model will embed/be able to learn. embedding_dim is the size of the continuous vector used to convert discrete word indices into continuous vector representations. The use of a vector representation makes it easier for a neural network to process and learn from text data.

## 3. Positional Encoding

***Why Postional Encoding is Needed:***
Since the transformer architecture doesn't have any inherent notion of sequence order (like RNNs or LSTMs do), we need to inject some information about the relative or absolute position of the words in the sequence.

***How Does Positional Encoding Work:***
A sinusoidal function is used to generate a position vector that can be recognized by the model. The sinusoidal nature of the function produces a different value for each word position (shown in graph below (run code)).

After the positional encoding is generated, it is added to the embedded vector. This process gives a vector that has information about the word and its position.

In [ ]:
# RUN CELL TO SEE VISUALIZATION

import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :].squeeze(1) # add positional encoding to embedded vector
        return x
    
posEnc = PositionalEncoding(512)
embVecs = torch.zeros(100, 512)
pe = posEnc.forward(embVecs)
plt.figure(figsize=(12, 8))
plt.pcolormesh(pe, cmap='viridis')
plt.xlabel('Embedding Dimensions')
plt.ylabel('Token Position')
plt.colorbar()
plt.title('Positional Encoding')
plt.show()

## 4. Query, Key, and Value Vectors
The attention mechanism can be thought of as a weighted sum of values, where the weight assigned to each value is determined by a compatibility function of the query with the corresponding key.

**1. Query (Q):** Represents the element we are currently focusing on. In the self-attention mechanism of transformers, the query comes from the same place as the key and value, which means it's derived from the input data.

**2. Key (K):** Represents all the elements in the input sequence. The role of the key is to be compared with the query to determine how much focus to place on the corresponding value.

**3. Value (V):** Represents all the elements in the input sequence. Once the weights are established using the keys and queries, these weights are used to sum the values to get the final self-attended representation of the input.

## 5. Multi-Head Attention

The attention weight between a query and a key is computed as the dot product of the query and the key, followed by a scaling factor and then a softmax operation to get the weights. The formula for computing the attention weights is:

**$$\text{Attention(Q, K, V) = softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$**

Query and Key values are first put through seperate fully connected neural networks. The outputs are then matrix-matrix multiplied (shown in function above). The matrix is then scaled by dividing the square root of the dimension. This is done to avoid exploding effects when multiplying values. Softmax function is then applied to allow the model to better understand what vectors to attend to; this yields the attention matrix. Finally the value vectors (which went through its own fully connected layer) is multiplied with the attention weights to get the output vectors.

The output vectors are concatenated and inputted into a linear layer.

This is called Multi-Head Attention because multiple sets of attention mechanisms are used in parallel. In theory each head learns to focus on different parts of the input. A good analogy is to think of it like several readers going through the same document. Each reader might focus on different details, but when you gather all their insights together, you get a more comprehensive understanding.

![MultiHeadAttention](images//MultiHeadAttention.png)

In [12]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        
        # Ensure the model size can be evenly divided among the attention heads
        assert d_model % num_heads == 0
        
        # Define depth of Q, K, V projections 
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.d_model = d_model
        
        # Define linear layers for Q, K, V projections
        self.W_Q = nn.Linear(d_model, d_model) 
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        
        # Linear layer for projecting the concatenated attention heads
        self.fc = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # Project input to Q, K, V for all heads (batch computation)
        # size(batch_size, num_heads, seq_length, d_k)
        Q = self.W_Q(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2) # we arent explicitly creating new layers for each head but
        K = self.W_K(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2) # rather using the same layer to project to different dimensions
        V = self.W_V(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2) # using different weights for each head
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        # attention shape: (batch_size, num_heads, seq_length, seq_length)
        attention = torch.nn.functional.softmax(scores, dim=-1)

        # Obtain weighted sum of V vectors
        context = torch.matmul(attention, V)
        
        # Concatenate heads and project result
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.fc(context)

d_model = 512  # Embedding size
seq_length = 10  # Example sequence length

input_seq = torch.randn(1, seq_length, d_model)
multi_head_attn = MultiHeadAttention(d_model=512, num_heads=8)
output = multi_head_attn(input_seq)

output.shape

torch.Size([1, 8, 10, 10])


torch.Size([1, 10, 512])

## 6. Residual Connections (Skip Connections)

A residual connection is a type of connection in neural networks where an input (or output) skips one or more layers. In the Transformer architecture, the output of the multihead attention layer is added to the original input. This is done to prevent the vanishing gradient problem and to improve convergence and generalization.
$$\text{Output} = F(x) + x$$

![ResidualConnection](images//ResidualConnection.png)

## 7. Layer Normalization
Layer normalization is a type of normalization technique that aims to stabalize training and improve generalization of a network. Layer normalization is similar to batch normalization but while batch normalization computes the mean and variance for each feature across a batch, layer normalization computes the mean and variance for each feature across the features' dimension (i.e., for each individual data point). This ensures that normalization is consistent across different batch sizes.

Layer normalization is applied on the feature dimension for each individual data point. Given $x$ as the input feature vector, the layer normalization is computed as:

**1. Calculate the mean:**
$$\mu = \frac{1}{d} \sum_{i=1}^{d} x_i$$
**2. Compute the variance:**
$$\sigma^2 = \frac{1}{d} \sum_{i=1}^{d} (x_i - \mu)^2$$
**3. Normalize the feature vector and scale and shift with learnable parameters $\gamma$ (scale) and $\beta$ (shift):**
$$\text{Norm}(x_i) = \gamma \left( \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \right) + \beta$$

In the context of Transformers, layer normalization is applied before each sub-layer, and the residual connection is added after the normalization. So, the sequence of operations is:

**1. Layer normalization on the input.**

**2. Apply the sub-layer (like multi-head attention).**

**3. Add the residual connection (i.e., the original input).**

In [ ]:
# Defining the TransformerBlock with Residual Connections and Layer Normalization

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads):
        super(TransformerBlock, self).__init__()
        
        # Multi-Head Attention layer
        self.attention = MultiHeadAttention(d_model, num_heads)
        
        # Feed-forward neural network (we'll use 4*d_model as the hidden layer size here)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        
        # Layer Normalization layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Dropout for regularization (optional, but we'll include here for demonstration)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        # Multi-Head Attention with residual connection and layer normalization
        attention_output = self.attention(x)
        x = self.norm1(x + self.dropout(attention_output))
        
        # Feed-forward neural network with residual connection and layer normalization
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

# Initialize the TransformerBlock
transformer_block = TransformerBlock(d_model=512, num_heads=8)

# For testing purposes, we'll create a dummy input and pass it through the TransformerBlock
dummy_input = torch.randn(1, 10, 512)
output_block = transformer_block(dummy_input)

output_block.shape


# Decoder 
The decoder generates the output sequence. It also has multiple identical layers with a structure similar to the encoder but with an additional multi-head attention layer.

## 1. Input

The input to the decoder block can vary based on whether the model is in the training phase or the evaluation (inference) phase. During training the decoder receives the target sequence, which is what we want the model to predict. For example, in a translation task, if we're translating French to English, the English sentence would be our target sequence. While the decoder uses this target sequence for its predictions, there's a special process called "masking" applied. This ensures that when predicting a particular word, the model can't "cheat" by looking ahead at future words in the target sequence. It's like asking someone to guess a word in a sentence without revealing the next word to them. masking occurs after scale is applied. However during inference the situation changes, because we don't have a target sequence (we are trying to generate that). So, the decoder starts with an initial word (often a start-of-sequence token) and then generates the subsequent words one by one. Each newly predicted word is then used as part of the input for the next prediction, in a step-by-step manner. This distinction in input handling ensures that the model learns correctly during training and operates effectively when used in real-world tasks.

## 2. Masked Multi-Headed self-Attention

In Masked Multi-Headed Self-Attention, the Attention Head is nearly identical to the encoder's however it has a masking function. Masking  is about ensuring predictions for a given token or word don't rely on future tokens in the sequence. This is important because during sequence generation, the future tokens are unknown. Imagine you're predicting a sentence word-by-word, and you're currently trying to predict the fifth word. If there was no masking, the attention mechanism could potentially "peek" at words six, seven, and so on. This doesn't align with real-world sequential generation where future words are yet to be determined.

![MaskedMultiHead](images//MaskedMultiHead.png)

Masking works by setting certain values in the attention weight matrix to very large negative numbers (-inf) (before the softmax operation). This ensures that the softmax output for these positions is effectively zero, which means the model gives no attention to these positions.

![maskScores](images//mask_score.png)

Masking is important because it ensures that the model will perform the same during training as it does during evaluation. For example, during training, the model has access to the full target sequence. But during inference, the model generates the output token-by-token. Masking ensures that the model's training aligns with how it will operate during inference. Additionally, masking maintains the temporal order of the data because the model cannot "look ahead".

In [ ]:
class MaskedMultiHeadAttention(MultiHeadAttention):
    def forward(self, x, mask=None):
        batch_size = x.size(0)
        
        Q = self.W_Q(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_K(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_V(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        
        # Apply the mask to scores before softmax
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attention = torch.nn.functional.softmax(scores, dim=-1)
        context = torch.matmul(attention, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        return self.fc(context)

# Function to create a mask for the attention scores
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    return mask.unsqueeze(0).unsqueeze(1)


seq_length = 10
mask = generate_square_subsequent_mask(seq_length)
masked_attention = MaskedMultiHeadAttention(d_model=512, num_heads=8)
dummy_input = torch.randn(1, seq_length, 512)  # Regenerating the dummy input
output_masked = masked_attention(dummy_input, mask=mask)

output_masked.shape


## 3. Residual Connection & Normalization

After the Masked Multihead Attention, the output is added to the original input of this block (this is the "residual connection"). Then, the result is normalized using layer normalization.


In [ ]:
class EncoderDecoderMultiHeadAttention(MultiHeadAttention):
    def forward(self, queries, keys, values, mask=None):
        # Assume keys and values come from encoder output, queries come from decoder output
        
        batch_size = queries.size(0)
        
        Q = self.W_Q(queries).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_K(keys).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_V(values).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attention = torch.nn.functional.softmax(scores, dim=-1)
        context = torch.matmul(attention, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        return self.fc(context)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads):
        super(DecoderBlock, self).__init__()
        
        # Masked Multihead Attention
        self.masked_attention = MaskedMultiHeadAttention(d_model, num_heads)
        
        # Encoder-Decoder Attention
        self.encoder_decoder_attention = EncoderDecoderMultiHeadAttention(d_model, num_heads)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x, encoder_output, mask=None):
        # Masked Multihead Attention
        masked_attention_output = self.masked_attention(x, mask=mask)
        #layer normalization and residual connection
        x = self.norm1(x + self.dropout(masked_attention_output))
        
        # Encoder-Decoder Attention
        attention_output = self.encoder_decoder_attention(x, encoder_output, encoder_output)
        x = self.norm2(x + self.dropout(attention_output))
        
        # Feed-forward Network
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))

        return self.softmax(x)

## 4. Multi-Head Self-Attention

After the masked self-attention, the decoder's output from the previous step is passed through another attention mechanism. This time, it attends to the output of the encoder. This step allows the decoder to focus on relevant parts of the original input sequence (as represented by the encoder's output) when generating each token in its output sequence. So the Encoder/Decoder Head gets the keys and values from the encoder block while getting the queries from the masked multi-head attention block.

In [ ]:
# NO NEED TO RUN THIS CELL

class EncoderDecoderMultiHeadAttention(MultiHeadAttention):
    def forward(self, queries, keys, values, mask=None):
        # Assume keys and values come from encoder output, queries come from decoder output
        
        batch_size = queries.size(0)
        
        Q = self.W_Q(queries).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_K(keys).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_V(values).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attention = torch.nn.functional.softmax(scores, dim=-1)
        context = torch.matmul(attention, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        return self.fc(context)


## 5. Feed-Forward Neural Network

The FFN is a simple neural network that exists within each encoder and decoder block of the Transformer architecture. Its primary purpose in the Transformer model is to perform additional transformations on the data after attention mechanisms have been applied, allowing the model to learn more complex representations. The FFN in the Transformer consists of two linear (dense) layers with a ReLU activation function in between. It doesn't have a recurrent or convolutional structure, which means it doesn't consider any order of the sequence. It applies the same operation to each position separately and identically.

The Feed-Forward Neural Network is can be seen in out decoder block pasted here:

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads):
        super(DecoderBlock, self).__init__()
        
        # Masked Multihead Attention
        self.masked_attention = MaskedMultiHeadAttention(d_model, num_heads)
        
        # Encoder-Decoder Attention
        self.encoder_decoder_attention = EncoderDecoderMultiHeadAttention(d_model, num_heads)
        
        # Feed-forward Network
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)

        # log softmax used for CrossEntropyLoss
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x, encoder_output, mask=None):
        # Masked Multihead Attention
        masked_attention_output = self.masked_attention(x, mask=mask)
        #layer normalization and residual connection
        x = self.norm1(x + self.dropout(masked_attention_output))
        
        # Encoder-Decoder Attention
        attention_output = self.encoder_decoder_attention(x, encoder_output, encoder_output)
        x = self.norm2(x + self.dropout(attention_output))
        
        # Feed-forward Network
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))

        # Softmax: get probabilities over next words in vocabulary
        return self.softmax(x)

## 6. Conclusion

We conclude by understanding its final output mechanism. At the end of the decoder's operations, for every position in our sequence, we obtain a probability distribution across the entire vocabulary of the model. It is like asking, "Given the context and the words I've seen so far, what's the likelihood of each word being the next one?"

From these probabilities, the word with the highest likelihood is chosen as the next word in our sequence. This process of selecting a new word will continue repeatedly, generating word after word until it decides it's complete (signaled by generating a special 'stop' or 'end-of-sequence' token).